In [2]:
!pip install -q python-gdcm
!pip install -q pylibjpeg
!pip install -q dicomsdl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 117.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.4 MB/s eta 0:00:00


In [17]:
!pip install pydicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.2 MB/s eta 0:00:00


In [3]:
import os,shutil, pandas as pd
from glob import glob
from tqdm import tqdm
import zipfile
import cv2
import numpy as np

In [11]:
from google.colab import drive
import os

In [14]:
ROOT_PATH = "/content/drive/MyDrive/rsna_data/"
IMG_DIR = "/content/drive/MyDrive/png_test_csm"

In [7]:
resize_dim = 512
img_size = [resize_dim, resize_dim]
idx = 0
parts = 1

## traindlfkd image label 합쳐주기

In [12]:
train_df = pd.read_csv(f'{ROOT_PATH}train.csv')
img_lvl_df = pd.read_csv(f'{ROOT_PATH}image_level_labels.csv')
train_df = train_df.merge(img_lvl_df, on=['patient_id'], how='right')

train_df['image_path'] = (f'{ROOT_PATH}train_images' + '/'
                          + train_df.patient_id.astype(str)
                          + '/' + train_df.series_id.astype(str)
                          + '/' + train_df.instance_number.astype(str) + '.dcm')

print('Train:')
print(f'# Size: {len(train_df)}')

Train:
# Size: 12029


## png로 바꾸기

In [15]:
#바꿔서 넣을 폴더 만들어주기
!rm -r {IMG_DIR}
os.makedirs(f'{IMG_DIR}/train_images', exist_ok = True)
os.makedirs(f'{IMG_DIR}/test_images', exist_ok = True)

In [18]:
import pydicom
import math
import cv2
import numpy as np

In [19]:
def standardize_pixel_array(dcm: pydicom.dataset.FileDataset) -> np.ndarray:
    """
    DICOM 이미지의 pixel_array를 보정합니다.

    Parameters:
    - dcm (pydicom.dataset.FileDataset): DICOM 이미지 데이터셋.

    Returns:
    - np.ndarray: 보정된 픽셀 배열.
    """
    # 만약 PixelRepresentation이 1이면 DICOM pixel_array를 보정

    pixel_array = dcm.pixel_array
    if dcm.PixelRepresentation == 1:
        bit_shift = dcm.BitsAllocated - dcm.BitsStored
        dtype = pixel_array.dtype
        new_array = (pixel_array << bit_shift).astype(dtype) >> bit_shift
        pixel_array = pydicom.pixel_data_handlers.util.apply_modality_lut(new_array, dcm)
    return pixel_array


In [20]:
def read_xray(path, fix_monochrome=True):
    """
    DICOM 이미지를 읽고 정규화한 후 반환

    Parameters:
    - path (str): DICOM 이미지 파일 경로.
    - fix_monochrome (bool, optional): 모노크롬 이미지를 고치는 옵션 (기본값: True).

    Returns:
    - np.ndarray: 정규화된 이미지 데이터.
    """
    # DICOM 파일을 읽어와서 픽셀 배열을 정규화
    dicom = pydicom.dcmread(path)
    data = standardize_pixel_array(dicom)
    data = data - np.min(data)
    data = data / (np.max(data) + 1e-5)

    # 모노크롬 이미지를 고치는 옵션을 고려
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = 1.0 - data
    return data

In [21]:
def resize_and_save(file_path):
    """
    DICOM 이미지를 리사이즈하고 PNG 파일로 저장한 후 관련 정보를 반환

    Parameters:
    - file_path (str): DICOM 이미지 파일 경로.

    Returns:
    - tuple: 환자 ID, 시리즈 ID, 이미지 ID, 원본 이미지 너비, 원본 이미지 높이.
    """
    img = read_xray(file_path)
    h, w = img.shape[:2]  # 원본 이미지 크기

    # 이미지 크롭
    min_dim = min(h, w)
    h_tail = (h - min_dim) // 2
    w_tail = (w - min_dim) // 2
    img = img[h_tail: h-h_tail, w_tail: w-w_tail]

    resize_dim = 256  # 리사이즈할 이미지 크기를 정의
    img = cv2.resize(img, (resize_dim, resize_dim), cv2.INTER_LINEAR)
    img = (img * 255).astype(np.uint16)

    # 이미지 파일의 경로를 분석하여 새 경로를 생성하고 저장
    sub_path = file_path.split("/", 4)[-1].split('.dcm')[0] + '.png'
    infos = sub_path.split('/')
    pid = infos[-3]
    sid = infos[-2]
    iid = infos[-1]
    iid = iid.replace('.png', '')
    new_path = os.path.join(IMG_DIR, sub_path)
    os.makedirs(new_path.rsplit('/', 1)[0], exist_ok=True)
    cv2.imwrite(new_path, img)

    return pid, sid, iid, w, h

train 데이터

In [23]:
from joblib import Parallel, delayed

size = -(-len(train_df) // parts)  # 데이터프레임의 크기를 파티션 수로 나눈 값 계산
file_paths = train_df.image_path.tolist()[idx*size:(idx+1)*size]  # 해당 파티션의 파일 경로 리스트를 얻음

# 병렬 처리를 사용하여 이미지 리사이즈 및 저장 작업을 수행
imgsize_train = Parallel(n_jobs=-1, backend='threading')(delayed(resize_and_save)(file_path)\
                                                         for file_path in tqdm(file_paths, leave=True, position=0))

 16%|█▌        | 1924/12029 [07:35<22:25,  7.51it/s]

KeyboardInterrupt: ignored

In [ ]:
path = train_df[train_df.width<700].image_path.iloc[0].replace(ROOT_PATH, IMG_DIR).replace('.dcm','.png')
img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
plt.figure(figsize=(10,10))
plt.imshow(img, cmap='gray')